# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Dingle,10.9995,122.6711,74.89,88,100,1.99,PH,1613082534
1,Zhuhai,22.2769,113.5678,62.60,72,20,12.66,CN,1613082534
2,Rikitea,-23.1203,-134.9692,79.09,71,82,8.52,PF,1613082534
3,Aykhal,66.0000,111.5000,-23.01,84,6,4.05,RU,1613082535
4,San Patricio,28.0170,-97.5169,41.00,93,90,17.27,US,1613082535
...,...,...,...,...,...,...,...,...,...
538,Santa Fe,-31.6333,-60.7000,84.00,50,83,1.99,AR,1613082359
539,Frolovo,49.7713,43.6622,33.01,92,100,6.33,RU,1613082629
540,Conde,-7.2597,-34.9075,80.60,74,0,12.66,BR,1613082629
541,Takelsa,36.7833,10.6333,53.60,62,20,5.75,TN,1613082630


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
63,Kibala,9.1104,18.3463,71.60,46,0,4.16,TD,1613082544
69,Ngunguru,-35.6167,174.5000,72.00,55,0,10.00,NZ,1613082545
239,São Filipe,14.8961,-24.4956,72.07,67,0,9.91,CV,1613082572
306,Nelson Bay,-32.7167,152.1500,73.99,73,0,6.91,AU,1613082582
314,Oussouye,12.4850,-16.5469,78.80,64,0,6.91,SN,1613082583
345,Tombouctou,20.0000,-3.0000,71.56,15,0,9.42,ML,1613082588
353,Salalah,17.0151,54.0924,71.60,53,0,3.44,OM,1613082589
356,Pilar,-9.5972,-35.9567,77.00,83,0,5.75,BR,1613082589
436,Jega,12.2175,4.3792,77.05,16,0,9.75,NG,1613082603
441,La Providencia,28.7167,-111.5833,79.00,34,0,10.00,MX,1613082607


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
63,Kibala,TD,9.1104,18.3463,
69,Ngunguru,NZ,-35.6167,174.5000,
239,São Filipe,CV,14.8961,-24.4956,
306,Nelson Bay,AU,-32.7167,152.1500,
314,Oussouye,SN,12.4850,-16.5469,
345,Tombouctou,ML,20.0000,-3.0000,
353,Salalah,OM,17.0151,54.0924,
356,Pilar,BR,-9.5972,-35.9567,
436,Jega,NG,12.2175,4.3792,
441,La Providencia,MX,28.7167,-111.5833,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 63: Kibala.
Closest hotel in Kibala is Hôtel des Chasses.
------------
Retrieving Results for Index 69: Ngunguru.
Closest hotel in Ngunguru is Quality Hotel Oceans Tutukaka.
------------
Retrieving Results for Index 239: São Filipe.
Closest hotel in São Filipe is La Fora Ecolodge.
------------
Retrieving Results for Index 306: Nelson Bay.
Closest hotel in Nelson Bay is The Anchorage Port Stephens Hotel & Spa.
------------
Retrieving Results for Index 314: Oussouye.
Closest hotel in Oussouye is Campement ALULUM.
------------
Retrieving Results for Index 345: Tombouctou.
Missing field/result... skipping.
------------
Retrieving Results for Index 353: Salalah.
Closest hotel in Salalah is HYATT SALALAH HOTEL.
------------
Retrieving Results for Index 356: Pilar.
Missing field/result... skipping.
------------
Retrieving Results for Index 436: Jega.
Closest hotel in Jega is Jega Guest Inn.
------------
Retrieving Results for Index 441: La Providencia.
Missing fie

In [9]:
# Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
63,Kibala,TD,9.1104,18.3463,Hôtel des Chasses
69,Ngunguru,NZ,-35.6167,174.5000,Quality Hotel Oceans Tutukaka
239,São Filipe,CV,14.8961,-24.4956,La Fora Ecolodge
306,Nelson Bay,AU,-32.7167,152.1500,The Anchorage Port Stephens Hotel & Spa
314,Oussouye,SN,12.4850,-16.5469,Campement ALULUM
345,Tombouctou,ML,20.0000,-3.0000,
353,Salalah,OM,17.0151,54.0924,HYATT SALALAH HOTEL
356,Pilar,BR,-9.5972,-35.9567,
436,Jega,NG,12.2175,4.3792,Jega Guest Inn
441,La Providencia,MX,28.7167,-111.5833,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))